In [1]:
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Cant remember why needed this ... 
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [ ]:

import pandas as pd
import json

# Import
diarized_df = pd.read_csv('./data/diarzed_output_named.csv')


In [ ]:
preprocessed_df = diarized_df.copy()
# Preprocessing steps for LDA analysis

preprocessed_df

,Unnamed: 0,start,end,text,words,speaker
0,0,21.467,23.068,"Hello, hello, and welcome.","[{'word': 'Hello,', 'start': 21.467, 'end': 21...",Clarkson
1,1,23.108,24.029,Thank you very much.,"[{'word': 'Thank', 'start': 23.108, 'end': 23....",Clarkson
2,2,24.049,34.256,"Now, as you know, the producers on this show l...","[{'word': 'Now,', 'start': 24.049, 'end': 24.1...",Clarkson
3,3,34.597,39.640,Then they set unbelievably hard tasks to do to...,"[{'word': 'Then', 'start': 34.597, 'end': 34.7...",Clarkson
4,4,39.900,40.080,Yeah.,"[{'word': 'Yeah.', 'start': 39.9, 'end': 40.08...",Hammond
...,...,...,...,...,...,...
1042,1042,269.865,271.447,We're through!,"[{'word': ""We're"", 'start': 269.865, 'end': 27...",Clarkson
1043,1043,273.209,277.514,"Both our cars were flooded, but our guides wer...","[{'word': 'Both', 'start': 273.209, 'end': 273...",SPEAKER_03
1044,1044,278.215,283.842,"People of Surrey, if this happens to you, you ...","[{'word': 'People', 'start': 278.215, 'end': 2...",SPEAKER_03
1045,1045,283.922,286.085,"Well, the people of Botswana have a tip for you.","[{'word': 'Well,', 'start': 283.922, 'end': 28...",SPEAKER_03


In [ ]:

preprocessed_df.to_csv("preprocess_df.csv")

In [ ]:
#### Below is a suggestion from gpt. 
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus import stopwords as nltk_stopwords

# 1. Remove emails, newline characters, and non-alphabetic characters
preprocessed_df['cleaned_text'] = preprocessed_df['text'].str.replace(r'\S+@\S+', '', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'http\S+|www\S+', '', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'\n', ' ', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# 2. Convert to lowercase
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.lower()

# 3. Tokenize the text
preprocessed_df['tokens'] = preprocessed_df['cleaned_text'].apply(lambda x: gensim.utils.simple_preprocess(x, deacc=True))

# 4. Remove stopwords
stopwords = set(nltk_stopwords.words('english'))
preprocessed_df['tokens'] = preprocessed_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

# 5. Lemmatize the tokens
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
preprocessed_df['lemmatized_tokens'] = preprocessed_df['tokens'].apply(lambda x: [token.lemma_ for token in nlp(" ".join(x)) if token.pos_ in ["NOUN", "ADJ", "VERB", "ADV"]])

# Display the preprocessed dataframe
preprocessed_df.head()
